In [4]:
import os
import pandas as pd
import numpy as np

print("Setup Complete")

Setup Complete


In [5]:
# LPF params:
# fs = 100.0
# ws = 12
# wp = 3
# wc = 4
# order = 4

b_z_low = [0.000177296607979, 0.000709186431917, 0.001063779647875, 0.000709186431917, 0.000177296607979]
a_z_low = [1, -3.349831562667920, 4.252610698953553, -2.420450670140820, 0.520508279582855]

# HPF params:
# fs = 100.0
# ws = 0.0005
# wp = 0.01
# wc = 0.005
# order = 2

b_z_high = [0.999777886079662, -1.999555772159325, 0.999777886079662]
a_z_high = [1, -1.999555722824731, 0.999555821493919]

# Apply filter using coefficients in z-domain
def apply_filter(data, b, a):
    data = np.asarray(data, dtype = np.float64)  # Input: float64
    y = np.zeros_like(data, dtype = np.float64)  # Output: float64
    for i in range(len(data)):
        y[i] = b[0] * data[i]
        for j in range(1, len(b)):
            if i-j >= 0:
                y[i] += b[j] * data[i-j]
        for j in range(1, len(a)):
            if i-j >= 0:
                y[i] -= a[j] * y[i-j]
    return y

In [6]:
input_folder = 'E:/mimic_dataset/mimic_perform_ppg_signal_csv_100hz'
output_folder = 'E:/mimic_dataset/mimic_perform_ppg_signal_csv_100hz_filtered'

os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        try:
            df = pd.read_csv(input_path, header=None)

            ppg_data = df[0].values

            low_pass_filtered = apply_filter(ppg_data, b_z_low, a_z_low)
            high_pass_filtered = apply_filter(low_pass_filtered, b_z_high, a_z_high)

            df_filtered = pd.DataFrame(high_pass_filtered)
            df_filtered.to_csv(output_path, index=False, header=False)

            print(f"Processed and filtered: {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

Processed and filtered: mimic_perform_af_001_data.csv
Processed and filtered: mimic_perform_af_002_data.csv
Processed and filtered: mimic_perform_af_003_data.csv
Processed and filtered: mimic_perform_af_004_data.csv
Processed and filtered: mimic_perform_af_005_data.csv
Processed and filtered: mimic_perform_af_006_data.csv
Processed and filtered: mimic_perform_af_007_data.csv
Processed and filtered: mimic_perform_af_008_data.csv
Processed and filtered: mimic_perform_af_009_data.csv
Processed and filtered: mimic_perform_af_010_data.csv
Processed and filtered: mimic_perform_af_011_data.csv
Processed and filtered: mimic_perform_af_012_data.csv
Processed and filtered: mimic_perform_af_013_data.csv
Processed and filtered: mimic_perform_af_014_data.csv
Processed and filtered: mimic_perform_af_015_data.csv
Processed and filtered: mimic_perform_af_016_data.csv
Processed and filtered: mimic_perform_af_017_data.csv
Processed and filtered: mimic_perform_af_018_data.csv
Processed and filtered: mimi